# First Order Predicate Logic: Forward chaining in Rule-Based Systems

### Task 0

In [1]:
from Lab08 import make_var, make_const, make_atom, make_or, make_neg, \
                is_variable, is_constant, is_atom, is_function_call, \
                print_formula, get_args, get_head, get_name, get_value ,\
                unify, substitute
from LPTester import *
from functools import reduce

def add_statement(kb, conclusion, *hypotheses):
    s = conclusion if not hypotheses else make_or(*([make_neg(s) for s in hypotheses] + [conclusion]))
    if check_sentence(s):
        kb.append(s)
        print("OK: Added statement " + print_formula(s, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(s, True)+"><" + str(s) + ">")
    return False

def assign_next_var_name():
    global var_no
    var_no += 1
    return "v" + str(var_no)

def gather_vars(S):
    return [get_name(S)] if is_variable(S) else \
        [] if not has_args(S) else reduce(lambda res, a: res + gather_vars(a), get_args(S), [])

def make_and(sentence1, sentence2, *others):
    return ['formula', 'and', [sentence1, sentence2] + list(others)]

dummy = make_atom("P")
[and_name, or_name, neg_name] = [get_head(s) for s in [make_and(dummy, dummy), make_or(dummy, dummy), make_neg(dummy)]]


def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [and_name, or_name]:
            return (len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S)) \
                and not [s for s in get_args(S) if not check_sentence(s)]
        if get_head(S) == neg_name:
            return (len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) \
                and check_sentence(get_args(S)[0])
    return pFail("Not sentence or unknown type", S)
def make_atom(predicate, *args):
    return ['formula', 'atom', predicate, list(args)]


def check_atom(A):
    if is_atom(A):
        return not [t for t in get_args(A) if not check_term(t)] and \
            (get_head(A) is not None or pFail("Predicate name is None", A))
    return pFail("Is not an atom", A)

def is_positive_literal(L):
    return is_atom(L)

def is_negative_literal(L):
    global neg_name
    return get_head(L) == neg_name and is_positive_literal(get_args(L)[0])

def make_unique_var_names(KB):
    global var_no
    var_no = 0
    return [substitute(S, {var: make_var(assign_next_var_name()) for var in gather_vars(S)}) for S in KB] 

def pFail(message, f):
    print(message + " <" + str(f) + ">")

def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))

>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
Test 24: OK
Test 25: OK
Test 26: OK
Test 27: OK
Test 28: OK
Test 29: OK
Test 30: OK
Test 31: OK
Test 32: OK
>>>  33 / 33 tests successful.
(and (or notP(?x) Q(?x)) T(?y, <built-in function add>[1,2]))
>>> Test batch [1]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
>>>  5 / 5 tests successful.
>>> Test batch [2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
>>>  7 / 7 tests successful.
>>> Test batch [3]
Test 0: OK, got < False >
Test 1: OK, got < {?x -> A} >
Test 2: OK, got < {?x -> A} >
Test 3: OK, got < {?x -> A} >
Test 4: OK, got < {?x -> A} >
Test 5: OK, got < {?x -> <built-in function add>[?Z,5]} >
Test 6: OK, got < {?z -> C, ?y -> B, ?x -> A} >
Tes

In [2]:
def make_const(value):
    return ['constant', value]

def make_var(name):
    return ['variable', name, None]

def make_function_call(function, *args):
    return ['function', function, list(args)]

def make_atom(predicate, *args):
    return ['formula', 'atom', predicate, list(args)]

def make_neg(sentence):
    return ['formula', 'not', [sentence]]

def make_and(sentence1, sentence2, *others):
    return ['formula', 'and', [sentence1, sentence2] + list(others)]

def make_or(sentence1, sentence2, *others):
    return ['formula', 'or', [sentence1, sentence2] + list(others)]

def replace_args(formula, new_args):
    if formula[0] == 'function':
        return ['function', formula[1], list(new_args)]
    elif formula[0] == 'formula':
        if formula[1] == 'atom':
            return ['formula', 'atom', formula[2], list(new_args)]
        else:
            return [formula[0], formula[1], list(new_args)]
    return formula

def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

def is_constant(f):
    return f[0] == 'constant'

def is_variable(f):
    return f[0] == 'variable'

def is_function_call(f):
    return f[0] == 'function'

def is_atom(f):
    return f[0] == 'formula' and f[1] == 'atom'

def is_sentence(f):
    return f[0] == 'formula'

def has_args(f):
    return is_function_call(f) or is_sentence(f)

def get_value(f):
    if is_constant(f):
        return f[1]
    return None

def get_name(f):
    if is_variable(f):
        return f[1]
    return None

def get_head(f):
    if is_function_call(f):
        return f[1]
    elif f[0] == 'formula':
        if is_atom(f):
            return f[2]
        else:
            return f[1]
    return None

def check_term(T):
    if is_constant(T):
        return (get_value(T) is not None) or pFail("The value of the constant is None", T)
    if is_variable(T):
        return (get_name(T) is not None) or pFail("The name of the variable is None", T)
    if is_function_call(T):
        return not [t for t in get_args(T) if not check_term(t)] and \
            (get_head(T) is not None or pFail("Function is not callable", T))
    return pFail("Term is not one of constant, variable or function call", T)

def get_args(f):
    if is_function_call(f):
        return f[2]
    elif is_atom(f):
        return f[3]
    elif is_sentence(f):
        return f[2]
    return None

In [4]:
def get_sports_kb():
    sports_kb = []
    add_statement(sports_kb, make_atom('Consecutive', make_const('Monday'), make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Tuesday'), make_const('Wednesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Wednesday'), make_const('Thursday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Thursday'), make_const('Friday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Friday'), make_const('Saturday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Saturday'), make_const('Sunday')))
    add_statement(sports_kb, make_atom('Weekend', make_const('Saturday')))
    add_statement(sports_kb, make_atom('Weekend', make_const('Sunday')))
    add_statement(sports_kb, make_atom('Rain', make_const('Friday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Monday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Wednesday')))
    add_statement(sports_kb, make_atom('Student', make_const('Mary')))
    add_statement(sports_kb, make_atom('Student', make_const('Kevin')))
    add_statement(sports_kb, make_atom('SummerSport', make_const('Volleyball')))
    add_statement(sports_kb, make_atom('WinterSport', make_const('Skiing')))
    add_statement(sports_kb, make_atom('WinterSport', make_const('Sledgging')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Sledgging')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Volleyball')))
    add_statement(sports_kb, make_atom('Activity', make_var('who'), make_var('what'), make_var('when')),
                  make_atom('GoesToMountain', make_var('who'), make_var('when')),
                  make_atom('PractiseSport', make_var('who'), make_var('what'))
                 )
    make_unique_var_names(sports_kb)
    return sports_kb


print("This is how the knowledge base is presented:")
skb = get_sports_kb()
print_KB(skb)
print("==================== \n Inside the KB we can notice:")
print("" + "".join([(str(s) + "\n") for s in skb]))

This is how the knowledge base is presented:
OK: Added statement Consecutive(Monday, Tuesday)
OK: Added statement Consecutive(Tuesday, Wednesday)
OK: Added statement Consecutive(Wednesday, Thursday)
OK: Added statement Consecutive(Thursday, Friday)
OK: Added statement Consecutive(Friday, Saturday)
OK: Added statement Consecutive(Saturday, Sunday)
OK: Added statement Weekend(Saturday)
OK: Added statement Weekend(Sunday)
OK: Added statement Rain(Friday)
OK: Added statement Sunny(Monday)
OK: Added statement Sunny(Tuesday)
OK: Added statement Sunny(Wednesday)
OK: Added statement Student(Mary)
OK: Added statement Student(Kevin)
OK: Added statement SummerSport(Volleyball)
OK: Added statement WinterSport(Skiing)
OK: Added statement WinterSport(Sledgging)
OK: Added statement PractiseSport(Kevin, Skiing)
OK: Added statement PractiseSport(Kevin, Sledgging)
OK: Added statement PractiseSport(Mary, Skiing)
OK: Added statement PractiseSport(Mary, Volleyball)
OK: Added statement (or notGoesToMountain

## Auxiliary functions

### Task 3

In [5]:
def get_premises(formula):
    premises = []
    for arg in get_args(formula):
        if is_negative_literal(arg):
            premises.append(*get_args(arg))
    return premises

def get_conclusion(formula):
    for arg in get_args(formula):
        if is_positive_literal(arg):
            return arg

def is_fact(formula):
    return is_positive_literal(formula)

def is_rule(formula):
    if get_head(formula) == 'or' or get_head(formula) == 'and':
        return True
    else:
        return False

f = make_or(make_neg(make_atom("P", make_var("x"))), make_neg(make_atom("Q", make_var("x"))), make_atom("R", make_var("x")))
print("".join([(print_formula(p, True) + " ; ") for p in get_premises(f)])[:-3]) # Should be P(?x) ; Q(?x)
print_formula(get_conclusion(f)) 
print(is_rule(f))
print(is_fact(get_conclusion(f))) 

P(?x) ; Q(?x)
R(?x)
True
True


In [6]:
def equal_terms(t1, t2):
    if is_constant(t1) and is_constant(t2):
        return get_value(t1) == get_value(t2)
    if is_variable(t1) and is_variable(t2):
            return get_name(t1) == get_name(t2)
    if is_function_call(t1) and is_function(t2):
        if get_head(t1) != get_head(t2):
            return all([equal_terms(get_args(t1)[i], get_args(t2)[i]) for i in range(len(get_args(t1)))])
    return False

def is_equal_to(a1, a2):
    if not (is_atom(a1) and is_atom(a2) and get_head(a1) == get_head(a2) and len(get_args(a1)) == len(get_args(a2))):
        return False
    return all([equal_terms(get_args(a1)[i], get_args(a2)[i]) for i in range(len(get_args(a1)))])

## Prove theorems by forward chaining

### Task 4

In [7]:
from copy import deepcopy

def get_premises(formula):
    premises = []
    for arg in get_args(formula):
        if is_negative_literal(arg):
            premises.append(arg)
    return premises

def apply_rule(rule, facts):
    result = []
    premises = []
    subst = []
    for i in get_premises(rule):
        premises.append(*get_args(i))
    for premise in premises:
        lista_subst = []
        for fact in facts:
            unification = unify(premise, fact)
            if unification != 0:
                subst.append(unification)
            for substitution in subst:
                unification = unify(premise, fact, substitution)
                if unification != 0:
                    lista_subst.append(unification)
        for new_subst in lista_subst:
            subst.append(new_subst)
    for sub in subst:
        substitution = substitute(get_conclusion(rule), sub)
        if substitution not in result and substitution!=0:
            result.append(substitution)

    return result

# Test!
# Rule: P(x) => Q(x)
# Facts: P(1)
for f in apply_rule( 
        make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))), \
        [make_atom("P", make_const(1))]):
    print_formula(f) # should be Q(1)
print("=====")
# Rule: P(x) ^ Q(x) => R(x)
# Facts: P(1), P(2), P(3), Q(3), Q(2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("x"))),
            make_atom("R", make_var("x"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]]):
    print_formula(f) # should be R(2) and R(3)
print("=====")
# Rule: P(x) ^ Q(y) ^ R(x, y) => T(x, y)
# Facts: P(1), P(2), P(3), Q(3), Q(2), R(3, 2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("y"))),
            make_neg(make_atom("R", make_var("x"), make_var("y"))),
            make_atom("T", make_var("x"), make_var("y"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]] + \
        [make_atom("R", make_const(3), make_const(2))]):
    print_formula(f) # should be T(3, 2)

Q(1)
=====
R(1)
R(2)
R(3)
=====
T(1, 3)
T(2, 3)
T(3, 3)
T(?x, 3)
T(3, 2)


In [8]:
def forward_chaining(kb, theorem, verbose = True):
    local_kb = deepcopy(kb)
    got_new_facts = True   
    is_proved = False    
    for fact in filter(is_fact, local_kb):
        if unify(fact, theorem):
            if verbose: print("This already in KB: " + print_formula(fact, True))
            is_proved = True
            break
    while (not is_proved) and got_new_facts:
        got_new_facts = False
        for rule in filter(is_rule, local_kb):
            # For each rule:
            new_facts = apply_rule(rule, list(filter(is_fact, local_kb)))
            new_facts = list(filter(lambda fact: not any(list(filter(lambda orig: is_equal_to(fact, orig), local_kb))), new_facts))
            if new_facts:
                if verbose: print("Applied rule: " + print_formula(rule, True))
                got_new_facts = True
                for fact in new_facts:
                    if unify(fact, theorem) != False:
                        is_proved = True
                        add_statement(local_kb, fact)
                        if verbose: print("Now in KB: " + print_formula(fact, True))
                        break
                    add_statement(local_kb, fact)
            if is_proved:
                break
    if verbose:
        if is_proved:
            print("The theorem is TRUE!")
        else:
            print("The theorem is FALSE!")
    return is_proved

In [9]:
def test_result(result, truth, idx):
    print("Test " + str(idx) + " OK!" if result == truth else "Test FAILED!")
    print("================== ")

test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_var("x")), True), True, 0)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_var("x")), True), True, 1)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_const("Thursday")), True), True, 2)
test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_const("Saturday")), True), True, 3)
test_result(forward_chaining(deepcopy(skb), make_atom("Activity",
                        make_const("Kevin"), make_var("Sport"), make_const("Saturday")), True), True, 4)

This already in KB: Sunny(Monday)
The theorem is TRUE!
Test 0 OK!
This already in KB: Rain(Friday)
The theorem is TRUE!
Test 1 OK!
Applied rule: (or notGoesToMountain(?who, ?when) notPractiseSport(?who, ?what) Activity(?who, ?what, ?when))
OK: Added statement Activity(Kevin, Skiing, ?when)
OK: Added statement Activity(Kevin, Sledgging, ?when)
OK: Added statement Activity(Mary, Skiing, ?when)
OK: Added statement Activity(Mary, Volleyball, ?when)
The theorem is FALSE!
Test FAILED!
Applied rule: (or notGoesToMountain(?who, ?when) notPractiseSport(?who, ?what) Activity(?who, ?what, ?when))
OK: Added statement Activity(Kevin, Skiing, ?when)
OK: Added statement Activity(Kevin, Sledgging, ?when)
OK: Added statement Activity(Mary, Skiing, ?when)
OK: Added statement Activity(Mary, Volleyball, ?when)
The theorem is FALSE!
Test FAILED!
Applied rule: (or notGoesToMountain(?who, ?when) notPractiseSport(?who, ?what) Activity(?who, ?what, ?when))
OK: Added statement Activity(Kevin, Skiing, ?when)
Now